In [1]:
# -*- coding: utf-8 -*-
from urllib2 import urlopen
from lxml import etree
import urllib
from lxml import html
from bs4 import BeautifulSoup
import io
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import numpy as np
import re
from collections import deque
import json
import urllib2
import time

In [2]:
def save_ad_phone_image(ad_id, ad_image_hash):
        ad_image_file = open('%s' % ad_id, "wb")
        ad_image_file.write(ad_image_hash.decode('base64'))
        ad_image_file.close()
        
class PageParser:
    
    """
    example page: https://www.avito.ru/moskva/tovary_dlya_kompyutera/nvidia_gtx_1080_amd_radeon_r9_270x_i_drugie_modeli_728193995
    extract:
        phone
        text description
        adress
        name
        price
        how to use: 
        >> parser = PageParser(url)
        >> parser.get_user_name()
    """
    
    description = None
    price = None
    def __init__(self, url, proxies, ip_cnt):
        self.url = url #!!!!
        self.page = urllib2.urlopen(self.url).read()
        self.soup = BeautifulSoup(self.page, 'html.parser')
        if proxies == None: 
            proxies = []
        self.proxies = proxies
        self.ip_usage = ip_cnt
    def urlopen(self, url):
        # open new ip each time using counter ip_usage
        prx = self.ip_usage % (len(self.proxies) + 1)
        self.ip_usage = (self.ip_usage + 1) % (len(self.proxies) + 1)
        if (prx == 0):
            print("usual ip")
            test_proxy = urllib2.ProxyHandler({})
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            return urllib2.urlopen(url)
        else:
            prx -= 1
            print self.proxies[prx]
            test_proxy = urllib2.ProxyHandler(self.proxies[prx])
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            answer = urllib2.urlopen(url)
            return answer
    """
    def find_price(self):
        return self.get_variable('avito.item.price')
    def get_title(self):
        tag_ = self.soup.find('div', class_="sticky-header-prop")
        return tag_.text.strip()
    def find_text_description(self):
        tag_ = self.soup.find('meta', attrs={"property" : "og:description"})
        self.description = tag_["content"]
        return self.description
    def find_id_number(self):
        tag_ = self.soup.find('div', class_='title-info-metadata-item')
        self.id_number = tag_.text
        self.id_number = self.get_number_from_str(self.id_number)
        return self.id_number
    def get_number_from_str(self, s):
        # переписать нормально мб
        while (not re.match('[0-9]', s[0])):
            s = s[1:]
        ans = ""
        i = 0
        while (re.match('[0-9]', s[i:])):
            i += 1
        return s[:i]
    # get image url by pkey and ID
    def get_image_pkey(self, ad_id=False, ad_phone=False):
        if ad_id and ad_phone:
            ad_subhash = re.findall(r'[0-9a-f]+', ad_phone)
            if int(ad_id) % 2 == 0:
                ad_subhash.reverse()
            ad_subhash = ''.join(ad_subhash)
            return ad_subhash[::3]
    def get_phone_json(self, id_, pkey_, sHash_, url_):
        req = "https://www.avito.ru/items/phone/{0}?pkey={1}&vsrc=r&searchHash={2}".format(id_, pkey_, sHash_)
        ref = 'https://www.avito.ru' + url_
        request = urllib2.Request(req, headers={'Referer': ref})
        response = self.urlopen(request)
        html = response.read()
        return html
    def get_variable(self, s):
        pre = re.compile(s + " = '(.*)';")    
        scripts = self.soup.find_all('script')
        answer = ""
        for i in scripts:
            try:
                ans = re.findall(pre, i.string)
                if len(ans) != 0:
                    answer = ans[0].strip()
            except:
                pass
        return answer   
    def get_ad_phone_hash(self):
        return self.get_variable('avito.item.phone')
    
    def get_avito_item_searchHash(self):
        return self.get_variable('avito.item.searchHash')
    
    def get_avito_item_id(self):
        return self.get_variable('avito.item.id')
    
    def get_user_name(self):
        # <div class="seller-info-name">
        tag_ = self.soup.find('div', class_='seller-info-name')
        return tag_.text.strip()
    
    def get_user_adress(self):
        tags = self.soup.find_all('div', class_='seller-info-prop')
        for i in tags:
            tg = i.find('div', class_='seller-info-label')
            if (tg.text == u'Адрес'):
                return i.find('div', class_='seller-info-value').text.strip()
        return "NONE"
    
    def get_phone_image(self):
        id_ = str(self.get_avito_item_id())
        phone_ = self.get_ad_phone_hash()
        pkey_ = self.get_image_pkey(id_, phone_)
        url_ = self.get_variable('avito.item.url')
        json = self.get_phone_json(id_, pkey_, self.get_avito_item_searchHash(), url_)
        self.json = json
        return self.json
    
    def save_image(self):
        ss = json.loads(self.get_phone_image())['image64']
        image = ss[ss.find(','):]
        save_ad_phone_image(parser.get_avito_item_id(), image)
    """
    def get_adress_region(self):
        tag_ = self.soup.find('span', itemprop="addressRegion")
        if tag_ == None:
            return None
        self.adress_region = tag_.text
        return self.adress_region
    def get_adress(self):
        tag_ = self.soup.find('span', itemprop="addressLocality")
        if tag_ is None:
            tag_ = self.soup.find('span', itemprop="address")
        if tag_ is None:
            return None
        self.adress = tag_.text
        return self.adress
    def get_telephone(self):
        tag_ = self.soup.find('span', itemprop="telephone")
        if tag_ is None:
            return None
        self.telephone = tag_.text
        return self.telephone
    def get_detail(self):
        tag_ = self.soup.find('p', class_="detail_company_doing")
        if tag_ is None:
            return None
        self.telephone = tag_.text
        return self.telephone

In [3]:
class Data:
    def __init__(self, proxies=None):
        """
        create a deque which is trying to get-and-scrap pages by url from Avito
        extracts data from pages via PageParser and return DataFrame object
        NOTE blocked if using with same ip non-stop more then 10 times (10 links)
        """
        self.index = 0
        self.links = deque()
        self.adresses = []
        self.telephones = []
        self.adress_regions = []
        self.details = []
        self.proxies = proxies
        self.ip_cnt = 0
    def add_link(self, lnk):
        self.links.append(lnk)
    def parse_next(self):
        self.ip_cnt += 1
        # full
        parser = PageParser(self.links[self.index], proxies=self.proxies, ip_cnt=self.ip_cnt)
        print("---")
        self.adresses.append(parser.get_adress())
        self.telephones.append(parser.get_telephone())
        self.adress_regions.append(parser.get_adress_region())
        self.details.append(parser.get_detail())
        self.index += 1
    def get_DataFrame(self):
        d = {'adress' : self.adresses, 'adress_regions' : self.adress_regions, 'telephon' : self.telephones, 'detail' : self.details}
        return pd.DataFrame(data=d)

In [9]:

lnk_counter = 0
url = ""
BEGIN = 'http://www.yp.ru'
data = Data(proxies=[{'http':'http://5.8.36.174:8085'}])
for big_page in range(5):
    url = "http://www.yp.ru/msk/search/text/табак"
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, 'html.parser')
    for item in soup.find_all("div", class_="search_item"):
        for s in item.find_all('p', class_="company_name"):

            lnk = s.find('a')['href']
            #  print "LINK: " +  lnk['href']
            #  print(lnk['title'])
            url_page = BEGIN + lnk
            print url_page
            # print(url_page)
            data.add_link(url_page)
            data.parse_next()
            break


        lnk_counter += 1
        if (lnk_counter > 1000):
            break
        print(lnk_counter)
        full_data = data
        #  print('---\n\n')
    tag_ = soup.find('li', class_='ui_next_page')
    print tag_
    tag_ = tag_.find('a')
    print(tag_)
    url = BEGIN + tag_['href']
    print('next big page')

http://www.yp.ru/msk/detail/id/delo_tabak_3388065/
---
1
http://www.yp.ru/msk/detail/id/sigary_tabak_trubki_stt_1502641/
---
2
http://www.yp.ru/msk/detail/id/tabak_plyus_1169905/
---
3
http://www.yp.ru/msk/detail/id/ves_tabak_magazin_2975469/
---
4
http://www.yp.ru/msk/detail/id/elit_tabak_treid_1270504/
---
5
http://www.yp.ru/msk/detail/id/ves_tabak_magazin_2975470/
---
6
http://www.yp.ru/msk/detail/id/ves_tabak_magazin_2975471/
---
7
http://www.yp.ru/msk/detail/id/tabak_1376229/
---
8
http://www.yp.ru/msk/detail/id/sherlock_magazin_tabaka_2556480/
---
9
http://www.yp.ru/msk/detail/id/mir_tabaka_magazin_tabachnykh_izdelii_2975479/
---
10
http://www.yp.ru/msk/detail/id/trubki_sigary_tabak_1299051/
---
11
http://www.yp.ru/msk/detail/id/tabak_1371055/
---
12
http://www.yp.ru/msk/detail/id/elit_tabak_magazin_2930155/
---
13
http://www.yp.ru/msk/detail/id/tabak_magazin_tabachnykh_izdelii_2930479/
---
14
http://www.yp.ru/msk/detail/id/galereya_tabaka_1260174/
---
15
http://www.yp.ru/msk/det

In [10]:
output = data.get_DataFrame()

In [11]:
output

,adress,adress_regions,detail,telephon
0,"140009, Московская область, Люберцы, Комсомоль...",Люберецкий район,\nДеятельность: \r\n ...,+7 (916) 641-33-73
1,"117209, Москва, Севастопольский проспект, дом ...",Юго-Западный АО,\nДеятельность: \r\n ...,+7 (499) 400-03-62
2,"125040, Москва, Ленинградский проспект, дом 7",None,None,+7 (495) 517-31-09
3,"117534, Москва, Кировоградская улица, дом 23-а",Южный АО,None,+7 (916) 185-49-78
4,"121059, Москва, Бережковская набережная, дом 2...",Западный АО,None,+7 (963) 711-35-96
5,"117405, Москва, Варшавское шоссе, дом 143-а, с...",Южный АО,None,+7 (916) 185-49-78
6,"117519, Москва, улица Красного Маяка, дом 10, ...",Южный АО,None,+7 (916) 185-49-78
7,"119002, Москва, Арбат улица, дом 36/2, строение 1",Центральный АО,None,+7 (495) 241-19-22
8,"111396, Москва, Зелёный проспект, дом 62, ТЦ Ш...",Восточный АО,None,+7 (495) 220-96-97
9,"125171, Москва, Ленинградское шоссе, дом 19",Северо-Западный АО,None,+7 (985) 199-56-92
